In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

In [91]:
data1 = pd.read_csv("train_lending_club.csv") # Modified dataset. 
data1

,issue_d,sub_grade,term,home_ownership,fico_range_low,total_acc,pub_rec,revol_util,annual_inc,int_rate,...,pub_rec_bankruptcies,addr_state,initial_list_status,fico_range_high,revol_bal,id,open_acc,emp_length,loan_status,time_to_earliest_cr_line
0,2014-01-01,D1,60 months,MORTGAGE,660.0,18.0,0.0,86.8,40440.0,16.99,...,0.0,AR,f,664.0,17264.0,11024793,11.0,2.0,1,478656.0
1,2014-01-01,C4,60 months,MORTGAGE,740.0,26.0,0.0,103.5,59000.0,15.61,...,0.0,VT,f,744.0,6725.0,10596078,8.0,3.0,1,541728.0
2,2014-01-01,A4,36 months,RENT,700.0,47.0,0.0,11.4,40000.0,7.90,...,0.0,TX,w,704.0,7613.0,10775616,27.0,1.0,1,657590.4
3,2014-01-01,D1,60 months,MORTGAGE,665.0,26.0,0.0,56.2,70000.0,16.99,...,0.0,IN,f,669.0,14173.0,10765610,9.0,11.0,1,328838.4
4,2014-01-01,C3,36 months,MORTGAGE,700.0,15.0,1.0,67.1,120000.0,14.98,...,0.0,LA,f,704.0,2549.0,10794837,8.0,2.0,1,305164.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236841,2015-12-01,C2,36 months,RENT,675.0,33.0,0.0,93.3,85000.0,12.59,...,0.0,FL,w,679.0,23969.0,67476992,27.0,0.0,0,276220.8
236842,2015-08-01,D2,36 months,MORTGAGE,665.0,15.0,0.0,17.8,72000.0,16.55,...,0.0,NJ,f,669.0,7918.0,56130981,8.0,11.0,0,533779.2
236843,2016-02-01,B1,36 months,MORTGAGE,710.0,19.0,0.0,61.2,72000.0,8.39,...,0.0,FL,w,714.0,16029.0,71502396,10.0,11.0,0,376185.6
236844,2016-06-01,C3,60 months,RENT,675.0,23.0,1.0,40.1,67000.0,13.99,...,1.0,MN,f,679.0,8422.0,83875883,10.0,11.0,1,867801.6


In [92]:
data2 = pd.read_csv("lending_club_loan_two.csv") # Original data (Kaggle)
data2

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,10000.0,36 months,11.44,329.48,B,B4,Marketing,10+ years,RENT,117000.0,...,16.0,0.0,36369.0,41.8,25.0,w,INDIVIDUAL,0.0,0.0,"0174 Michelle Gateway\r\nMendozaberg, OK 22690"
1,8000.0,36 months,11.99,265.68,B,B5,Credit analyst,4 years,MORTGAGE,65000.0,...,17.0,0.0,20131.0,53.3,27.0,f,INDIVIDUAL,3.0,0.0,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113"
2,15600.0,36 months,10.49,506.97,B,B3,Statistician,< 1 year,RENT,43057.0,...,13.0,0.0,11987.0,92.2,26.0,f,INDIVIDUAL,0.0,0.0,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113"
3,7200.0,36 months,6.49,220.65,A,A2,Client Advocate,6 years,RENT,54000.0,...,6.0,0.0,5472.0,21.5,13.0,f,INDIVIDUAL,0.0,0.0,"823 Reid Ford\r\nDelacruzside, MA 00813"
4,24375.0,60 months,17.27,609.33,C,C5,Destiny Management Inc.,9 years,MORTGAGE,55000.0,...,13.0,0.0,24584.0,69.8,43.0,f,INDIVIDUAL,1.0,0.0,"679 Luna Roads\r\nGreggshire, VA 11650"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396025,10000.0,60 months,10.99,217.38,B,B4,licensed bankere,2 years,RENT,40000.0,...,6.0,0.0,1990.0,34.3,23.0,w,INDIVIDUAL,0.0,0.0,"12951 Williams Crossing\r\nJohnnyville, DC 30723"
396026,21000.0,36 months,12.29,700.42,C,C1,Agent,5 years,MORTGAGE,110000.0,...,6.0,0.0,43263.0,95.7,8.0,f,INDIVIDUAL,1.0,0.0,"0114 Fowler Field Suite 028\r\nRachelborough, ..."
396027,5000.0,36 months,9.99,161.32,B,B1,City Carrier,10+ years,RENT,56500.0,...,15.0,0.0,32704.0,66.9,23.0,f,INDIVIDUAL,0.0,0.0,"953 Matthew Points Suite 414\r\nReedfort, NY 7..."
396028,21000.0,60 months,15.31,503.02,C,C2,"Gracon Services, Inc",10+ years,MORTGAGE,64000.0,...,9.0,0.0,15704.0,53.8,20.0,f,INDIVIDUAL,5.0,0.0,"7843 Blake Freeway Apt. 229\r\nNew Michael, FL..."


In [93]:
data1.info() # Modified dataset. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236846 entries, 0 to 236845
Data columns (total 27 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   issue_d                   236846 non-null  object 
 1   sub_grade                 236846 non-null  object 
 2   term                      236846 non-null  object 
 3   home_ownership            236846 non-null  object 
 4   fico_range_low            236846 non-null  float64
 5   total_acc                 236846 non-null  float64
 6   pub_rec                   236846 non-null  float64
 7   revol_util                236846 non-null  float64
 8   annual_inc                236846 non-null  float64
 9   int_rate                  236846 non-null  float64
 10  dti                       236846 non-null  float64
 11  purpose                   236846 non-null  object 
 12  mort_acc                  236846 non-null  float64
 13  loan_amnt                 236846 non-null  f

In [94]:
data2.info() # Original data (Kaggle)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396030 entries, 0 to 396029
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   loan_amnt             396030 non-null  float64
 1   term                  396030 non-null  object 
 2   int_rate              396030 non-null  float64
 3   installment           396030 non-null  float64
 4   grade                 396030 non-null  object 
 5   sub_grade             396030 non-null  object 
 6   emp_title             373103 non-null  object 
 7   emp_length            377729 non-null  object 
 8   home_ownership        396030 non-null  object 
 9   annual_inc            396030 non-null  float64
 10  verification_status   396030 non-null  object 
 11  issue_d               396030 non-null  object 
 12  loan_status           396030 non-null  object 
 13  purpose               396030 non-null  object 
 14  title                 394274 non-null  object 
 15  

In [95]:
def get_similar_column_names(data1 : pd.DataFrame, data2 : pd.DataFrame):
    
    similar_column_names = []

    for i, column in enumerate(list(data1.columns)):
        if column in list(data2.columns):
            similar_column_names.append(column)

    return similar_column_names

In [96]:
def get_non_similar_columns_names(data1 : pd.DataFrame, data2 : pd.DataFrame, similar_column_names : list):

    non_similar_names_2 = [name for name in list(data2.columns) if name not in similar_column_names]
    non_similar_names_1 = [name for name in list(data1.columns) if name not in similar_column_names]

    return non_similar_names_1, non_similar_names_2

In [97]:
# Retrieve all same names
similar_column_names = []
for i, column in enumerate(list(data1.columns)):
    if column in list(data2.columns):
        similar_column_names.append(column)

In [98]:
non_similar_names_2 = [name for name in list(data2.columns) if name not in similar_column_names]
non_similar_names_1 = [name for name in list(data1.columns) if name not in similar_column_names]

In [99]:
# Datasets have the same num of features, but only 22 which have the same names
# We should take a look on the other 5 features to understand what're their purpose, is it important to keep them etc. 

In [100]:
data1[non_similar_names_1]

,fico_range_low,addr_state,fico_range_high,id,time_to_earliest_cr_line
0,660.0,AR,664.0,11024793,478656.0
1,740.0,VT,744.0,10596078,541728.0
2,700.0,TX,704.0,10775616,657590.4
3,665.0,IN,669.0,10765610,328838.4
4,700.0,LA,704.0,10794837,305164.8
...,...,...,...,...,...
236841,675.0,FL,679.0,67476992,276220.8
236842,665.0,NJ,669.0,56130981,533779.2
236843,710.0,FL,714.0,71502396,376185.6
236844,675.0,MN,679.0,83875883,867801.6


In [101]:
data2[non_similar_names_2]

,grade,emp_title,title,earliest_cr_line,address
0,B,Marketing,Vacation,Jun-1990,"0174 Michelle Gateway\r\nMendozaberg, OK 22690"
1,B,Credit analyst,Debt consolidation,Jul-2004,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113"
2,B,Statistician,Credit card refinancing,Aug-2007,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113"
3,A,Client Advocate,Credit card refinancing,Sep-2006,"823 Reid Ford\r\nDelacruzside, MA 00813"
4,C,Destiny Management Inc.,Credit Card Refinance,Mar-1999,"679 Luna Roads\r\nGreggshire, VA 11650"
...,...,...,...,...,...
396025,B,licensed bankere,Debt consolidation,Nov-2004,"12951 Williams Crossing\r\nJohnnyville, DC 30723"
396026,C,Agent,Debt consolidation,Feb-2006,"0114 Fowler Field Suite 028\r\nRachelborough, ..."
396027,B,City Carrier,pay off credit cards,Mar-1997,"953 Matthew Points Suite 414\r\nReedfort, NY 7..."
396028,C,"Gracon Services, Inc",Loanforpayoff,Nov-1990,"7843 Blake Freeway Apt. 229\r\nNew Michael, FL..."


In [102]:
# address == addr_state 
# earliest_cr_line (datetime) == time_to_earliest_cr_line (timestamp)
# title seems to refer to the purpose of the loan. Seems interesting to keep it Only present in data2. Maybe predict it for data1
# emp_title seems to refer to the employment title.  Seems interesting to keep it Only present in data2. Maybe predict it for data1
# grade == subgrade // subgrade is more detailed, we don't have to keep grade, subgrade seems to be good. 
# [fico_range_low, fico_range_high] seems to bring more information but only present in data1. Features can maybe be predict for data2
# id looks like the id of the loan and is only present in data1. We will see if it ccan be the primary key for both of our datasets. 

In [103]:
data1[similar_column_names]

,issue_d,sub_grade,term,home_ownership,total_acc,pub_rec,revol_util,annual_inc,int_rate,dti,...,loan_amnt,application_type,installment,verification_status,pub_rec_bankruptcies,initial_list_status,revol_bal,open_acc,emp_length,loan_status
0,2014-01-01,D1,60 months,MORTGAGE,18.0,0.0,86.8,40440.0,16.99,15.16,...,17775.0,Individual,441.66,Verified,0.0,f,17264.0,11.0,2.0,1
1,2014-01-01,C4,60 months,MORTGAGE,26.0,0.0,103.5,59000.0,15.61,16.74,...,29175.0,Individual,703.45,Verified,0.0,f,6725.0,8.0,3.0,1
2,2014-01-01,A4,36 months,RENT,47.0,0.0,11.4,40000.0,7.90,20.34,...,6000.0,Individual,187.75,Not Verified,0.0,w,7613.0,27.0,1.0,1
3,2014-01-01,D1,60 months,MORTGAGE,26.0,0.0,56.2,70000.0,16.99,23.15,...,15600.0,Individual,387.62,Verified,0.0,f,14173.0,9.0,11.0,1
4,2014-01-01,C3,36 months,MORTGAGE,15.0,1.0,67.1,120000.0,14.98,17.88,...,10000.0,Individual,346.56,Verified,0.0,f,2549.0,8.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236841,2015-12-01,C2,36 months,RENT,33.0,0.0,93.3,85000.0,12.59,23.93,...,21000.0,Individual,703.44,Source Verified,0.0,w,23969.0,27.0,0.0,0
236842,2015-08-01,D2,36 months,MORTGAGE,15.0,0.0,17.8,72000.0,16.55,20.48,...,14000.0,Individual,496.01,Not Verified,0.0,f,7918.0,8.0,11.0,0
236843,2016-02-01,B1,36 months,MORTGAGE,19.0,0.0,61.2,72000.0,8.39,16.87,...,10000.0,Individual,315.17,Not Verified,0.0,w,16029.0,10.0,11.0,0
236844,2016-06-01,C3,60 months,RENT,23.0,1.0,40.1,67000.0,13.99,22.03,...,12000.0,Individual,279.16,Source Verified,1.0,f,8422.0,10.0,11.0,1


In [104]:
data2[similar_column_names]

,issue_d,sub_grade,term,home_ownership,total_acc,pub_rec,revol_util,annual_inc,int_rate,dti,...,loan_amnt,application_type,installment,verification_status,pub_rec_bankruptcies,initial_list_status,revol_bal,open_acc,emp_length,loan_status
0,Jan-2015,B4,36 months,RENT,25.0,0.0,41.8,117000.0,11.44,26.24,...,10000.0,INDIVIDUAL,329.48,Not Verified,0.0,w,36369.0,16.0,10+ years,Fully Paid
1,Jan-2015,B5,36 months,MORTGAGE,27.0,0.0,53.3,65000.0,11.99,22.05,...,8000.0,INDIVIDUAL,265.68,Not Verified,0.0,f,20131.0,17.0,4 years,Fully Paid
2,Jan-2015,B3,36 months,RENT,26.0,0.0,92.2,43057.0,10.49,12.79,...,15600.0,INDIVIDUAL,506.97,Source Verified,0.0,f,11987.0,13.0,< 1 year,Fully Paid
3,Nov-2014,A2,36 months,RENT,13.0,0.0,21.5,54000.0,6.49,2.60,...,7200.0,INDIVIDUAL,220.65,Not Verified,0.0,f,5472.0,6.0,6 years,Fully Paid
4,Apr-2013,C5,60 months,MORTGAGE,43.0,0.0,69.8,55000.0,17.27,33.95,...,24375.0,INDIVIDUAL,609.33,Verified,0.0,f,24584.0,13.0,9 years,Charged Off
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396025,Oct-2015,B4,60 months,RENT,23.0,0.0,34.3,40000.0,10.99,15.63,...,10000.0,INDIVIDUAL,217.38,Source Verified,0.0,w,1990.0,6.0,2 years,Fully Paid
396026,Feb-2015,C1,36 months,MORTGAGE,8.0,0.0,95.7,110000.0,12.29,21.45,...,21000.0,INDIVIDUAL,700.42,Source Verified,0.0,f,43263.0,6.0,5 years,Fully Paid
396027,Oct-2013,B1,36 months,RENT,23.0,0.0,66.9,56500.0,9.99,17.56,...,5000.0,INDIVIDUAL,161.32,Verified,0.0,f,32704.0,15.0,10+ years,Fully Paid
396028,Aug-2012,C2,60 months,MORTGAGE,20.0,0.0,53.8,64000.0,15.31,15.88,...,21000.0,INDIVIDUAL,503.02,Verified,0.0,f,15704.0,9.0,10+ years,Fully Paid


In [105]:
# loan status is stored with different format in both of dataset. We should reformat that.  
# application type is in the same format, but write differently. We should reformat it. 
# Term gives the same information but is in the wrong format. We should reformat it. 
# issue_d gives the same information in both dataset but is first in the wrong format and also the issue date is more detailed in data1. We will reformat and keep only month and year. 
# emp_lenght gives the same information in both whenever in data1 it's more detailed and don't have the same format. We will categorized it, respecting the format on data2. 
# After the dates, we will arrange the id so it can be the primary key of our dataset. 

In [106]:
data2 = data2.drop(columns=["grade"]) # Dropping grade because subgrade is more precised. 

In [107]:
data1["time_to_earliest_cr_line"] = pd.to_datetime(data1["time_to_earliest_cr_line"], unit='s', origin=pd.Timestamp('1960-01-01')) # Changing date format
data1["time_to_earliest_cr_line"]

0        1960-01-06 12:57:36.000000000
1        1960-01-07 06:28:48.000000000
2        1960-01-08 14:39:50.399999976
3        1960-01-04 19:20:38.399999976
4        1960-01-04 12:46:04.800000012
                      ...             
236841   1960-01-04 04:43:40.800000012
236842   1960-01-07 04:16:19.199999988
236843   1960-01-05 08:29:45.600000024
236844   1960-01-11 01:03:21.600000024
236845   1960-01-04 07:36:28.800000012
Name: time_to_earliest_cr_line, Length: 236846, dtype: datetime64[ns]

In [108]:
data2["earliest_cr_line"] = pd.to_datetime(data2["earliest_cr_line"], format="mixed")
data2["earliest_cr_line"]

0        1990-06-01
1        2004-07-01
2        2007-08-01
3        2006-09-01
4        1999-03-01
            ...    
396025   2004-11-01
396026   2006-02-01
396027   1997-03-01
396028   1990-11-01
396029   1998-09-01
Name: earliest_cr_line, Length: 396030, dtype: datetime64[ns]

In [109]:
data1 = data1.rename(columns={"time_to_earliest_cr_line" : "earliest_cr_line", "addr_state": "state"})

In [110]:
data2["state"] = [re.findall("[A-Z][A-Z]", data2["address"][i])[0] for i in range(len(list(data2["address"])))]
data2 = data2.drop(columns=["address"])

In [111]:
# We are going to merge data1 into data2

In [112]:
to_replace = list(data2["emp_length"].unique())
replace = [10, 4, 0, 6, 9, 2, 3, 8, 7, 5, 1, None]

for i in range(len(replace)):
    data2["emp_length"] = data2["emp_length"].replace(to_replace[i], replace[i])

data2["emp_length"] = data2["emp_length"].astype("category")

C:\Users\axelo\AppData\Local\Temp\ipykernel_1568\675339886.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data2["emp_length"] = data2["emp_length"].replace(to_replace[i], replace[i])


In [113]:
to_replace = list(data2["loan_status"].unique())
for i in range(0, 2):
    data2["loan_status"] = data2["loan_status"].replace(to_replace[i], i)

data2["loan_status"] = data2["loan_status"].astype("category")

C:\Users\axelo\AppData\Local\Temp\ipykernel_1568\3677252787.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data2["loan_status"] = data2["loan_status"].replace(to_replace[i], i)


In [114]:
data2["term"] = [re.findall("[0-9][0-9]", data2["term"][i])[0] for i in range(len(list(data2["term"])))]
data2["term"] = data2["term"].astype("int64")

In [115]:
data1["term"] = [re.findall("[0-9][0-9]", data1["term"][i])[0] for i in range(len(list(data1["term"])))]
data1["term"] = data2["term"].astype("int64")

In [116]:
data2["issue_d"] = pd.to_datetime(data2["issue_d"], format="mixed")
data1["issue_d"] = pd.to_datetime(data1["issue_d"], format="mixed")

In [117]:
data1["application_type"] = data1["application_type"].str.upper()
data1["application_type"] = data1["application_type"].replace(data1["application_type"].unique()[1], data2["application_type"].unique()[1])

In [61]:
IDs = []
data1_id = set((data1['id']))

for i in range(data2.__len__()):

    a = random.randint(10000000, 99999999)
    temp = a
    
    while (a in data1_id) or (a in IDs):
        a = random.randint(10000000, 99999999)
    
    IDs.append(a)

In [118]:
data2["id"] = IDs

In [119]:
data1

,issue_d,sub_grade,term,home_ownership,fico_range_low,total_acc,pub_rec,revol_util,annual_inc,int_rate,...,pub_rec_bankruptcies,state,initial_list_status,fico_range_high,revol_bal,id,open_acc,emp_length,loan_status,earliest_cr_line
0,2014-01-01,D1,36,MORTGAGE,660.0,18.0,0.0,86.8,40440.0,16.99,...,0.0,AR,f,664.0,17264.0,11024793,11.0,2.0,1,1960-01-06 12:57:36.000000000
1,2014-01-01,C4,36,MORTGAGE,740.0,26.0,0.0,103.5,59000.0,15.61,...,0.0,VT,f,744.0,6725.0,10596078,8.0,3.0,1,1960-01-07 06:28:48.000000000
2,2014-01-01,A4,36,RENT,700.0,47.0,0.0,11.4,40000.0,7.90,...,0.0,TX,w,704.0,7613.0,10775616,27.0,1.0,1,1960-01-08 14:39:50.399999976
3,2014-01-01,D1,36,MORTGAGE,665.0,26.0,0.0,56.2,70000.0,16.99,...,0.0,IN,f,669.0,14173.0,10765610,9.0,11.0,1,1960-01-04 19:20:38.399999976
4,2014-01-01,C3,60,MORTGAGE,700.0,15.0,1.0,67.1,120000.0,14.98,...,0.0,LA,f,704.0,2549.0,10794837,8.0,2.0,1,1960-01-04 12:46:04.800000012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236841,2015-12-01,C2,60,RENT,675.0,33.0,0.0,93.3,85000.0,12.59,...,0.0,FL,w,679.0,23969.0,67476992,27.0,0.0,0,1960-01-04 04:43:40.800000012
236842,2015-08-01,D2,36,MORTGAGE,665.0,15.0,0.0,17.8,72000.0,16.55,...,0.0,NJ,f,669.0,7918.0,56130981,8.0,11.0,0,1960-01-07 04:16:19.199999988
236843,2016-02-01,B1,60,MORTGAGE,710.0,19.0,0.0,61.2,72000.0,8.39,...,0.0,FL,w,714.0,16029.0,71502396,10.0,11.0,0,1960-01-05 08:29:45.600000024
236844,2016-06-01,C3,36,RENT,675.0,23.0,1.0,40.1,67000.0,13.99,...,1.0,MN,f,679.0,8422.0,83875883,10.0,11.0,1,1960-01-11 01:03:21.600000024


In [120]:
data2["issue_d"]

0        2015-01-01
1        2015-01-01
2        2015-01-01
3        2014-11-01
4        2013-04-01
            ...    
396025   2015-10-01
396026   2015-02-01
396027   2013-10-01
396028   2012-08-01
396029   2010-06-01
Name: issue_d, Length: 396030, dtype: datetime64[ns]

In [121]:
data1["issue_d"]

0        2014-01-01
1        2014-01-01
2        2014-01-01
3        2014-01-01
4        2014-01-01
            ...    
236841   2015-12-01
236842   2015-08-01
236843   2016-02-01
236844   2016-06-01
236845   2015-05-01
Name: issue_d, Length: 236846, dtype: datetime64[ns]

In [122]:
data2.to_csv("data2.csv")
data1.to_csv("data1.csv")

##  SEMI SUPERVISED LEARNING

In [129]:
data_1 = pd.read_csv("data1.csv")
data_2 = pd.read_csv("data2.csv")

batch_data2= pd.read_csv("data2.csv", skiprows=lambda x: x > 0 and random.random() > 0.2)  # Keeps  7% of rows randoml
batch_data2 = batch_data2.drop(columns = ["Unnamed: 0"])
batch_data2 = batch_data2.dropna()

numeric_target_1 = data_1["fico_range_high"]
numeric_target_2 = data_1["fico_range_low"]
categorical_target_1 = batch_data2["emp_title"]
categorical_target_2 = batch_data2["title"]

data_2 = data_2.drop(columns = ["emp_title", "title"])
data_2 = data_2.dropna()
batch_data2 = batch_data2[get_similar_column_names(data1, batch_data2)]
data_2 = data_2.drop(columns=["Unnamed: 0"])

In [70]:
get_non_similar_columns_names(data1, batch_data2, get_similar_column_names(data1, batch_data2))

(['fico_range_low', 'fico_range_high'], [])

In [71]:
def get_numeric_models():
    
    reg = LinearRegression()
    dt = DecisionTreeRegressor()
    rf = RandomForestRegressor()
    xg = xgb.XGBRegressor()
    knn = KNeighborsRegressor()
    nn = MLPRegressor()

    models = {"Linear Regressor": reg, "Decision Tree Regressor": dt, "Random Forest Regressor": rf, "XG Boost Regressor": xg, "Neural Network Regressor": nn, "K-Neirest Neighbors Regressor": knn}

    return models

In [72]:
def get_classifiers():
    
    lr = LogisticRegression()
    dc = DecisionTreeClassifier()
    rc = RandomForestClassifier()
    knc = KNeighborsClassifier()

    classifiers = {"Logistic Regressor": lr, "Decision Tree Classifier": dc, "Random Forest Classifier": rc, "K-Neirest Neighbors Classifier": knc}
    return classifiers

In [73]:
def prepare_data_for_training(data : pd.DataFrame):

    copy_data = data.copy()
    num_columns = [column for column in list(copy_data.columns) if (copy_data[column].dtype == "float64" or copy_data[column].dtype == "int64")]
    cat_columns = ["sub_grade", "home_ownership", "purpose", "initial_list_status", "application_type", "verification_status", "state"]
    le = LabelEncoder()
    scaler = StandardScaler()

    # Transforming columns 
    copy_data["issue_d"] = pd.to_datetime(copy_data["issue_d"], format="mixed")
    copy_data["earliest_cr_line"] = pd.to_datetime(copy_data["earliest_cr_line"], format="mixed")
    
    copy_data["issue_d_ts"] = copy_data["issue_d"].astype("int64") // 10**9
    copy_data["earliest_cr_line_ts"] = copy_data["earliest_cr_line"].astype("int64") // 10**9

    for column in num_columns:
        copy_data[f"{column}"] = scaler.fit_transform(copy_data[[column]])
    
    for column in cat_columns:
        copy_data[column] = le.fit_transform(copy_data[column])
    

    # Dropping useless columns
    copy_data = copy_data.drop(columns=["id"])
    copy_data = copy_data.drop(columns=["issue_d"])
    copy_data = copy_data.drop(columns=["earliest_cr_line"])

    return copy_data

In [74]:
def train_model_numeric(data : pd.DataFrame, models : dict, target : pd.Series):

    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)
    
    acc_dict = {}

    for model_name in models.keys():
        models[model_name].fit(X_train, y_train)
        score = models[model_name].score(X_test, y_test)
        print(f"Score for {model_name} : {score}")
        acc_dict[models[model_name]] = score
    
    max_acc = max(list(acc_dict.values()))
    
    for model, acc in list(acc_dict.items()):
        if acc ==  max_acc:
            saved_model =  model
    
    return saved_model

In [75]:
def train_model_categorical(data : pd.DataFrame, models : dict, target : pd.Series):

    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)
    
    acc_dict = {}

    for model_name in models.keys():
        models[model_name].fit(X_train, y_train)
        score = models[model_name].score(X_test, y_test)
        print(f"Score for {model_name} : {score}")
        acc_dict[models[model_name]] = score
    
    max_acc = max(list(acc_dict.values()))
    
    for model, acc in list(acc_dict.items()):
        if acc ==  max_acc:
            saved_model =  model
    
    return saved_model

In [76]:
def apply_prediction_numerical(model, data_point : pd.Series):

    data_point = data_point.to_frame().T
    prediction = model.predict(prepare_data_for_training(data_point).to_numpy()) 
    
    return prediction


In [77]:
def apply_prediction_categorical(model, data_point : pd.Series):

    data_point = data_point.to_frame().T
    prediction = model.predict(prepare_data_for_training(data_point).to_numpy()) 
    
    return prediction
    

### Prediction of numerical values

In [78]:
copy_data_1 = data1.copy()
copy_data_1 = copy_data_1.drop(columns=["fico_range_high", "fico_range_low"])
numeric_X = prepare_data_for_training(copy_data_1)

In [79]:
numeric_model_1 = train_model_numeric(numeric_X, get_numeric_models(), numeric_target_1)
numeric_model_2 = train_model_numeric(numeric_X, get_numeric_models(), numeric_target_2)

Score for Linear Regressor : 0.40898232334161777
Score for Decision Tree Regressor : 0.0782839769309619
Score for Random Forest Regressor : 0.548700715709499
Score for XG Boost Regressor : 0.5708629871611848
Score for Neural Network Regressor : -58165.2939162217
Score for K-Neirest Neighbors Regressor : -0.10263742585529778
Score for Linear Regressor : 0.40898760083753494
Score for Decision Tree Regressor : 0.07744960877710383
Score for Random Forest Regressor : 0.5482025702110183
Score for XG Boost Regressor : 0.5718697932071506
Score for Neural Network Regressor : -260983.68751768555
Score for K-Neirest Neighbors Regressor : -0.10263331567928469


In [80]:
apply_prediction_numerical(numeric_model_1, batch_data2.iloc[1])

array([695.61346], dtype=float32)

In [81]:
apply_prediction_numerical(numeric_model_2, batch_data2.iloc[1])

array([689.6517], dtype=float32)

In [132]:
prediction_fico_range_high = []
prediction_fico_range_low = []

for i in range(data_2.__len__()):
    prediction_fico_range_high.append(apply_prediction_numerical(numeric_model_1, data_2.iloc[i])[0])
    prediction_fico_range_low.append(apply_prediction_numerical(numeric_model_2, data_2.iloc[i])[0])

In [133]:
data_2["fico_range_high"] = prediction_fico_range_high
data_2["fico_range_low"] = prediction_fico_range_low

In [150]:
data_1 = data_1.drop(columns=["Unnamed: 0"])
data_2 = data_2.drop(columns=["Unnamed: 0"])

In [141]:
len(data_1.columns.to_list()) == len(data_2.columns.to_list())

True

In [142]:
data_2.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'sub_grade',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'purpose', 'dti', 'earliest_cr_line',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'application_type', 'mort_acc',
       'pub_rec_bankruptcies', 'state', 'id', 'fico_range_high',
       'fico_range_low'],
      dtype='object')

In [143]:
data_1.columns

Index(['issue_d', 'sub_grade', 'term', 'home_ownership', 'fico_range_low',
       'total_acc', 'pub_rec', 'revol_util', 'annual_inc', 'int_rate', 'dti',
       'purpose', 'mort_acc', 'loan_amnt', 'application_type', 'installment',
       'verification_status', 'pub_rec_bankruptcies', 'state',
       'initial_list_status', 'fico_range_high', 'revol_bal', 'id', 'open_acc',
       'emp_length', 'loan_status', 'earliest_cr_line'],
      dtype='object')

### Prediction for categorical values

In [144]:
# We will just drop those 2 categrorical variables because training the models take too much time

In [145]:
# batch_data2_copy = batch_data2.copy()
# categorical_X = prepare_data_for_training(batch_data2)

In [146]:
# cat_model_2 = train_model_categorical(categorical_X, get_classifiers(), categorical_target_2)

In [147]:
data_2.to_csv("data2.csv")
data_1.to_csv("data1.csv")

## MERGING

In [151]:
data_2 = pd.read_csv("data2.csv")
data_1 = pd.read_csv("data1.csv")

In [152]:
data_1 = data_1.drop(columns=["Unnamed: 0"])
data_2 = data_2.drop(columns=["Unnamed: 0"])

In [153]:
merged_data = pd.concat([data_1, data_2], sort=False) 

In [154]:
merged_data.columns

Index(['issue_d', 'sub_grade', 'term', 'home_ownership', 'fico_range_low',
       'total_acc', 'pub_rec', 'revol_util', 'annual_inc', 'int_rate', 'dti',
       'purpose', 'mort_acc', 'loan_amnt', 'application_type', 'installment',
       'verification_status', 'pub_rec_bankruptcies', 'state',
       'initial_list_status', 'fico_range_high', 'revol_bal', 'id', 'open_acc',
       'emp_length', 'loan_status', 'earliest_cr_line'],
      dtype='object')

In [155]:
merged_data.__len__()

577621

In [156]:
# columns to predict : emp_title, title, fico_range_high, fico_range_low

In [157]:
merged_data

,issue_d,sub_grade,term,home_ownership,fico_range_low,total_acc,pub_rec,revol_util,annual_inc,int_rate,...,pub_rec_bankruptcies,state,initial_list_status,fico_range_high,revol_bal,id,open_acc,emp_length,loan_status,earliest_cr_line
0,2014-01-01,D1,36,MORTGAGE,660.00000,18.0,0.0,86.8,40440.0,16.99,...,0.0,AR,f,664.00000,17264.0,11024793,11.0,2.0,1,1960-01-06 12:57:36.000000000
1,2014-01-01,C4,36,MORTGAGE,740.00000,26.0,0.0,103.5,59000.0,15.61,...,0.0,VT,f,744.00000,6725.0,10596078,8.0,3.0,1,1960-01-07 06:28:48.000000000
2,2014-01-01,A4,36,RENT,700.00000,47.0,0.0,11.4,40000.0,7.90,...,0.0,TX,w,704.00000,7613.0,10775616,27.0,1.0,1,1960-01-08 14:39:50.399999976
3,2014-01-01,D1,36,MORTGAGE,665.00000,26.0,0.0,56.2,70000.0,16.99,...,0.0,IN,f,669.00000,14173.0,10765610,9.0,11.0,1,1960-01-04 19:20:38.399999976
4,2014-01-01,C3,60,MORTGAGE,700.00000,15.0,1.0,67.1,120000.0,14.98,...,0.0,LA,f,704.00000,2549.0,10794837,8.0,2.0,1,1960-01-04 12:46:04.800000012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340770,2013-03-01,B4,36,RENT,682.20660,9.0,0.0,97.1,64000.0,13.11,...,0.0,SC,w,695.70280,11456.0,99200167,7.0,5.0,0,1991-11-01
340771,2015-10-01,B4,60,RENT,684.61163,23.0,0.0,34.3,40000.0,10.99,...,0.0,DC,w,695.52716,1990.0,85385934,6.0,2.0,0,2004-11-01
340772,2015-02-01,C1,36,MORTGAGE,681.01620,8.0,0.0,95.7,110000.0,12.29,...,0.0,LA,f,691.82745,43263.0,45032336,6.0,5.0,0,2006-02-01
340773,2013-10-01,B1,36,RENT,679.64370,23.0,0.0,66.9,56500.0,9.99,...,0.0,NY,f,695.84600,32704.0,45084163,15.0,10.0,0,1997-03-01


In [158]:
merged_data.to_csv("merged_data.csv")